<a href="https://colab.research.google.com/github/ds4geo/ds4geo/blob/master/DS4GEO_L3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import numpy as np
from scipy import interpolate

In [ ]:
air_temp = pd.read_excel("https://github.com/ds4geo/ds4geo/raw/master/data/timeseries/Au%C3%9Fenluft%2BEingangslabyrinth.xlsx", sheet_name="Daten3", usecols=[0,6], names=["time", "air_temp"])

In [ ]:
air_temp

In [ ]:
cave_temp = pd.read_excel("https://github.com/ds4geo/ds4geo/raw/master/data/timeseries/Au%C3%9Fenluft%2BEingangslabyrinth.xlsx", sheet_name="Daten3", usecols=[8,9], names=["time", "cave_temp"])

In [ ]:
cave_temp

In [ ]:
cave_temp.plot("time", "cave_temp")
air_temp.plot("time", "air_temp")

In [ ]:
cave_co2 = pd.read_excel(r"https://github.com/ds4geo/ds4geo/raw/master/data/timeseries/CO2%20_compiled.xlsx",
                         sheet_name="Data Stream (2)", names=["time", "unknown", "d13C", "d18O", "ppm CO2", "ctime"])

In [ ]:
cave_co2

In [ ]:
fig, ax = plt.subplots(figsize=(18,10))
ax2 = ax.twinx()
air_temp.plot("time", "air_temp", ax=ax)
cave_temp.plot("time", "cave_temp", ax=ax)
cave_co2.plot("time", "ppm CO2", ax=ax2, color="r")
ax.set_xlim([datetime.date(2015,9,17), datetime.date(2015,11,20)])

In [ ]:
# get data only in range
air_temp.index = air_temp.time # make time the index
air_select = air_temp["2015-9-17" : "2015-11-20"]

cave_temp.index = cave_temp.time
cave_select = cave_temp["2015-9-17" : "2015-11-20"]

cave_co2.index = cave_co2.time
co2_select = cave_co2["2015-9-17" : "2015-11-20"]

In [ ]:
# Check lengths
print("air data shape:", air_select.shape)
print("cave data shape:", cave_select.shape)
print("co2 data shape:", co2_select.shape)
# All different

In [ ]:
# Check sampling rate/timing
print("air sampling rate:",(air_select.index[1:] - air_select.index[:-1]).value_counts())
print("cave sampling rate:",(cave_select.index[1:] - cave_select.index[:-1]).value_counts())
print("co2 sampling rate:",(co2_select.index[1:] - co2_select.index[:-1]).value_counts())
# air is 1 hour, cave temp is 15 mins, co2 is all over the place. Mostly around 5 minutes, but sometimes 10, or much longer.
# If we resample to the cave temp 15 mins, then we are upsampling air temp, and downsampling the co2 data

In [ ]:
# Testing upsampling of air temps
kinds = ["linear", "nearest", "zero", "slinear", "quadratic", "cubic"]#, "previous", "next"]
intp_air = []
for k in kinds:
  interpf = interpolate.interp1d(air_select.time.to_numpy().astype(float), air_select.air_temp, kind=k, fill_value="extrapolate")
  air_interp = interpf(cave_select.time.to_numpy().astype(float))
  intp_air.append(air_interp)

In [ ]:
fig, ax = plt.subplots(figsize=(15,10))
for i,l in zip(intp_air, kinds):
  ax.plot(cave_select.index, i, "x-", label=l)
ax.plot(air_select.index, air_select.air_temp, "o")
ax.set_xlim(datetime.date(2015,10,15), datetime.date(2015,10,16))
ax.set_ylim(-4, 3)
ax.legend()

In [ ]:
# Downsampling of co2 data
kinds = ["linear", "nearest", "zero", "slinear", "quadratic", "cubic"]#, "previous", "next"]
intp_co2 = []
for k in kinds:
  interpf = interpolate.interp1d(co2_select.time.to_numpy().astype(float), co2_select["ppm CO2"], kind=k, fill_value="extrapolate")
  air_interp = interpf(cave_select.time.to_numpy().astype(float))
  intp_co2.append(air_interp)

In [ ]:
fig, ax = plt.subplots(figsize=(15,10))
ax.plot(co2_select.index, co2_select["ppm CO2"], "o")
for i,l in zip(intp_co2, kinds):
  ax.plot(cave_select.index, i, "x-", label=l)
#ax.set_xlim(datetime.date(2015,10,17), datetime.date(2015,10,18))
ax.set_xlim(datetime.datetime(2015,10,17, 9), datetime.datetime(2015,10,17,15))
ax.set_ylim(425, 465)
ax.legend()
# alternative downsampling to be addressed while filtering in next section - e.g. mean of window, decimate, etc.

In [ ]:
# Now interpolated, can do direct comparisons:
fig, ax = plt.subplots(figsize=(15,10))
ax.scatter(intp_co2[0], intp_air[0]-cave_select.cave_temp, marker="x")
ax.set_xlim([375, 450])

In [ ]:
cave_select.cave_temp.shape

In [ ]:
# Smoothing - need relevant data set to smooth!
#scipy-cookbook.readthedocs.io/items/SignalSmooth.html

In [ ]:
xdat = pd.read_csv("https://doi.pangaea.de/10.1594/PANGAEA.859980?format=textfile", header=48, sep="\t")


In [ ]:
xdat

In [ ]:
fig, ax1 = plt.subplots(figsize=(12,8))
ax2 = ax1.twinx()
ax3 = ax1.twinx()
ax4 = ax1.twinx()
xdat.plot("Depth [m]", "Al [cps]", ax=ax1, color="C0")
xdat.plot("Depth [m]", "K [cps]", ax=ax2, color="C1")
xdat.plot("Depth [m]", "Ti [cps]", ax=ax3, color="C2")
xdat.plot("Depth [m]", "V [cps]", ax=ax4, color="C3")
ax1.set_xlim(0.4,0.5)

In [ ]:
xdat.columns

In [ ]:
filter_length = 5
filter = np.ones(filter_length ) / filter_length

In [ ]:
filter

In [ ]:
Vf = np.convolve(xdat["V [cps]"],filter)

In [ ]:
Vf

In [ ]:
%matplotlib notebook
fig, ax = plt.subplots(figsize=(12,8))
ax.plot(xdat["Depth [m]"], Vf[2:-2])
ax.plot(xdat["Depth [m]"], xdat["V [cps]"])
ax.set_xlim(0.4,0.5)
plt.show()

In [ ]:
'''
Rule of thumb:
Matplotlib for static graphics, quick plotting and publication quality graphics
Plotly for interactive or static/publication quality
Bokeh for interactive/web plots
Seaborn for quick but powerful stats plots
Altair - different approach to making plots based on how we reason about data

'''
# Lets try bokeh
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

# Call once to configure Bokeh to display plots inline in the notebook.
output_notebook()

In [ ]:
p = figure()
p.line(xdat["Depth [m]"], Vf[2:-2])
p.line(xdat["Depth [m]"], xdat["V [cps]"])
show(p)